Summarizing, you must first scale (or standardize) separately each input variable
of your data, scale your desired output variables, and use these scaled patterns to train
the neural network. Then, when the training is finished, you can make predictions over
new patterns in three steps: scale the pattern, introduce it into the neural network,
and descale the obtained output from the network.

Import out in-house neural network class, at the beginning it will output a lot of internal debugging immediately

In [ ]:
from NeuralNet import NeuralNet

In [ ]:
layers = [4, 9, 5, 1]
nn = NeuralNet(layers=layers)

print("L = ", nn.L, end="\n")
print("n = ", nn.n, end="\n")

print("xi = ", nn.xi, end="\n")
print("xi[0] = ", nn.xi[0], end="\n")
print("xi[1] = ", nn.xi[0], end="\n")

print("wh = ", nn.w, end="\n")
print("wh[1] = ", nn.w[1], end="\n")

L =  4
n =  [4, 9, 5, 1]
xi =  [array([0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]), array([0.])]
xi[0] =  [0. 0. 0. 0.]
xi[1] =  [0. 0. 0. 0.]
wh =  [array([[0.]]), array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0.]])]
wh[1] =  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
    # Optional: Plot the evolution of the training and validation errors
    # Feed−forward all test patterns
    # Descale the predictions of test patterns, and evaluate them
    # test_predictions = self.predict(x)